In [ ]:
# General imports
import numpy as np
np.random.seed(42)
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib notebook
from matplotlib import colors as mcolors
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
import sys
import os
from resum.utilities import plotting_utils as plotting
from resum.multi_fidelity_gaussian_process import MultiFidelityVisualizer
from resum.multi_fidelity_gaussian_process import MFGPModel
from resum.multi_fidelity_gaussian_process import InequalityConstraints

In [ ]:
version = 'v1.6'
file_in=f'Ge77_rates_CNP_{version}.csv'
if not os.path.exists(f'out/mfgp/{version}'):
   os.makedirs(f'out/mfgp/{version}')
if not os.path.exists(f'in/mfgp/Ge77_rates_new_samples_{version}.csv'):
   fout = open(f'in/mfgp/Ge77_rates_new_samples_{version}.csv')
   fout.write("#\n ,Sample,Mode,Radius[cm],Thickness[cm],NPanels,Theta[deg],Length[cm],Ge-77[nevents],Ge-77_CNP,Ge-77_CNP_err")
   fout.close()
   

# Set parameter name/x_labels -> needs to be consistent with data input file
x_labels=['Radius[cm]','Thickness[cm]','NPanels', 'Theta[deg]', 'Length[cm]']
x_labels_out = ['Radius [cm]','Thickness [cm]','NPanels', 'Angle [deg]', 'Length [cm]']
y_label_cnp = 'y_cnp'
y_err_label_cnp = 'y_cnp_err'
y_label_sim = 'y_rGe77[nuc/(kg*yr)]'

# Set parameter boundaries
xmin=[0,0,0,0,0]
xmax=[265,20,360,90,150]
x_fixed = [160, 2, 40, 45, 20]

# Set parameter boundaries for aquisition function
parameters={'Radius[cm]': [90,250], 'Thickness[cm]':[2,15],'NPanels': [4,360], 'Theta[deg]': [0,90], 'Length[cm]': [1,150]}
parameters_draw={'Radius [cm]': [0,265],'Thickness [cm]': [0,20],'NPanels': [0,360], 'Angle [deg]': [0,90], 'Length [cm]': [0,150]}

# Assign costs
low_fidelity_cost = 1.
high_fidelity_cost = 2000.


In [ ]:
data=pd.read_csv(f'in/mfgp/{file_in}')

LF_cnp_noise=np.mean(data.loc[data['Fidelity']==0.][y_err_label_cnp].to_numpy())
HF_cnp_noise=np.mean(data.loc[data['Fidelity']==1.][y_err_label_cnp].to_numpy())

x_train_l, x_train_h, y_train_l, y_train_h = ([],[],[],[])
row_h=data.index[data['Fidelity'] == 1].tolist()
row_l=data.index[data['Fidelity'] == 0].tolist()

x_train_hf_sim = data.loc[data['Fidelity']==1.][x_labels].to_numpy().tolist()
y_train_hf_sim = data.loc[data['Fidelity']==1.][y_label_sim].to_numpy().tolist()

x_train_hf_cnp = data.loc[data['Fidelity']==1.][x_labels].to_numpy().tolist()
y_train_hf_cnp = data.loc[data['Fidelity']==1.][ y_label_cnp].to_numpy().tolist()

x_train_lf_sim = data.loc[data['Fidelity']==0.][x_labels].to_numpy().tolist()
y_train_lf_sim = data.loc[data['Fidelity']==0.][ y_label_sim].to_numpy().tolist()

x_train_lf_cnp = data.loc[data['Fidelity']==0.][x_labels].to_numpy().tolist()
y_train_lf_cnp = data.loc[data['Fidelity']==0.][ y_label_cnp].to_numpy().tolist()

trainings_data = {"lf": [x_train_lf_cnp,y_train_lf_cnp], "mf": [x_train_hf_cnp,y_train_hf_cnp], "hf": [x_train_hf_sim,y_train_hf_sim]}
noise = {"lf": LF_cnp_noise, "mf": 0., "hf": 0.}


In [ ]:
class NeutronModeratorInequalityConstraints(InequalityConstraints):
    def __init__(self, get_inner_radius, get_outer_radius, is_crossed):
        self.get_inner_radius = get_inner_radius
        self.get_outer_radius = get_outer_radius
        self.is_crossed = is_crossed

    def evaluate(self, x):
        super().evaluate(x)
        delta_x = np.ones(len(x))
        for i, xi in enumerate(x[:, :-1]):
            if self.get_inner_radius(xi) < 90.0:
                delta_x[i] = 0.0
            elif self.get_outer_radius(xi) > 265.0:
                delta_x[i] = 0.0
            elif self.get_outer_radius(xi) - self.get_inner_radius(xi) > 20.0:
                delta_x[i] = 0.0
            elif (
                xi[2] * xi[1] * xi[4]
                > 1.05 * np.pi * (self.get_outer_radius(xi)**2 - self.get_inner_radius(xi)**2)
            ):
                delta_x[i] = 0.0
            elif self.is_crossed(xi):
                delta_x[i] = 0.0
            else:
                delta_x[i] = 1.0
        return delta_x[:, None]

In [ ]:
inequalities = NeutronModeratorInequalityConstraints(plotting.get_inner_radius, plotting.get_outer_radius, plotting.is_crossed)

In [ ]:
mf_model = MFGPModel(trainings_data,noise, inequalities)
mf_model.build_model(1)

In [ ]:
mf_model.model.gpy_model

In [ ]:
%%capture
leg_label = []
ncol=3
nrow=int(np.ceil(len(x_labels)/ncol))
fig2,_  = plt.subplots(nrow, ncol, figsize=(15, 5), constrained_layout=True)
fig4,_  = plt.subplots(int(np.ceil(len(x_labels)/1)), 1, figsize=(5, 12), constrained_layout=True)



In [ ]:
mf_vis = MultiFidelityVisualizer(mf_model,parameters_draw,x_fixed)

In [ ]:

data_new=pd.read_csv(f'in/mfgp/Ge77_rates_new_samples_{version}.csv')
sample=0
add_new_sample=True
mf_model.set_traings_data(trainings_data)

while ( sample <= data_new['Sample'].max()):
    print('Sample #', sample)

    if sample > 0:    
         
        x_train_hf_sim = data_new[(data_new['Fidelity']==1.) & (data_new['Sample']==sample)][x_labels].to_numpy().tolist()
        y_train_hf_sim = data_new[(data_new['Fidelity']==1.) & (data_new['Sample']==sample)][y_label_sim].to_numpy().tolist()
        print(f"Adding {x_train_hf_sim}")   
        x_train_hf_cnp = data_new[(data_new['Fidelity']==1.) & (data_new['Sample']==sample)][x_labels].to_numpy().tolist()
        y_train_hf_cnp = data_new[(data_new['Fidelity']==1.) & (data_new['Sample']==sample)][y_label_cnp].to_numpy().tolist()

        x_train_lf_cnp = data_new[(data_new['Fidelity']==0.) & (data_new['Sample']==sample)][x_labels].to_numpy().tolist()
        y_train_lf_cnp = data_new[(data_new['Fidelity']==0.) & (data_new['Sample']==sample)][ y_label_cnp].to_numpy().tolist()
        trainings_data_new = {"lf": [x_train_lf_cnp,y_train_lf_cnp], "mf": [x_train_hf_cnp,y_train_hf_cnp], "hf": [x_train_hf_sim,y_train_hf_sim]}
        mf_model.set_data(trainings_data_new)


    # run the model drawing
    fig1, ax = plt.subplots(2, 3,figsize=(15, 5),constrained_layout=True)
    fig1 = mf_vis.draw_model_projections(fig1)
    fig2 = mf_vis.draw_model_projections(fig2)

    # find the next data point
    x_next_sample, us_acquisition = mf_model.max_acquisition_integrated_variance_reduction(parameters)
    print(f'next suggested point to simulated is at: {x_next_sample}')
    fig4 = mf_vis.draw_acquisition_func(fig4, us_acquisition, np.array(x_next_sample))
    sample+=1




In [ ]:
fig4.savefig("out/mfgp/model.png")

In [ ]:
%matplotlib inline
fig2.show()
fig4.show()

In [ ]:
fig3 = mf_vis.draw_model()

In [ ]:
data_test=pd.read_csv(f'in/mfgp/hf_validation_data_v1.2.csv')
x_test = data_test.loc[data_test['Fidelity']==1.][x_labels].to_numpy().tolist()
y_test = data_test.loc[data_test['Fidelity']==1.][y_label_sim].to_numpy().tolist()
fig5, validation  = mf_vis.model_validation(x_test, y_test)
plt.show()
